In [27]:
!pip install transformers
!pip install scikit-learn
!pip install pandas
!pip install numpy
!pip install tensorflow-gpu


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf

from collections import defaultdict
from sklearn.model_selection import train_test_split
from transformers import TFBertModel, BertTokenizerFast, create_optimizer
from transformers import AutoTokenizer, DataCollatorWithPadding, TFAutoModelForSequenceClassification, AutoConfig, BertModel, RobertaForSequenceClassification, TFRobertaForSequenceClassification, TFXLMRobertaForSequenceClassification
from transformers import create_optimizer

epochs = 1
batch = 16
nrows = None
idx = None

PATH_TO_LABEL_TIME = 'data/output_time.csv'
PATH_TO_LABEL_TOP_100 = 'data/output.csv'

PATH_TO_FEATURE_VECTOR = 'data/features/output.csv'
PATH_TO_FEATURE_RECEIVED_BY = 'data/features/output_received_by.csv'
PATH_TO_FEATURE_TITLE = 'data/features/output_title.csv'
PATH_TO_FEATURE_OMK = 'data/features/output_omk.csv'
PATH_TO_TEXT = 'data/output_heavy.csv'

In [29]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

def get_layer_for_texts(MAX_SEQUENCE_LENGTH=512):

    base = TFXLMRobertaForSequenceClassification.from_pretrained("models/XLM-RoBERTa")

    # Inputs for token indices and attention masks
    input_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='text.input_ids')
    attention_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='text.attention_mask')

    for layer in base.layers:
        layer.trainable = False

    # output = base([input_ids, attention_mask]).last_hidden_state[:, 0, :]
    output = base([input_ids, attention_mask])[0]
    output = tf.keras.layers.Dropout(rate=0.1)(output)


    return input_ids, attention_mask, output

def tokenize_texts(sentences, max_length=512, padding='max_length'):
    return tokenizer(
        sentences,
        truncation=True,
        padding=padding,
        max_length=max_length,
        return_tensors="tf"
    )

def data(batch_size=batch):

    # Load labels
    df_time = pd.read_csv(PATH_TO_LABEL_TIME, dtype={'id': int, 'label': str, 'label_encoded': int}, sep=',', na_values='')
    df_responsible = pd.read_csv(PATH_TO_LABEL_TOP_100, dtype={'id': int, 'label': str, 'label_encoded': int}, sep=',', na_values='')

    # Load input features
    df_feature_vector = pd.read_csv(PATH_TO_FEATURE_VECTOR, sep=',', na_values='')
    df_feature_received_by = pd.read_csv(PATH_TO_FEATURE_RECEIVED_BY, sep=',', na_values='')
    df_feature_title = pd.read_csv(PATH_TO_FEATURE_TITLE, sep=',', na_values='')
    df_feature_omk = pd.read_csv(PATH_TO_FEATURE_OMK, sep=',', na_values='')

    # Load input text
    df_texts = pd.read_csv(PATH_TO_TEXT, sep=',')
    df_texts = df_texts.rename(columns={'requestId': 'id'})
    df_texts['text'] = df_texts['subject'] + " " + df_texts['description']

    # Fill NaN just in case
    df_time = df_time.fillna('')
    df_responsible = df_responsible.fillna('')
    df_feature_vector = df_feature_vector.fillna('')
    df_feature_received_by = df_feature_received_by.fillna('')
    df_feature_title = df_feature_title.fillna('')
    df_feature_omk = df_feature_omk.fillna('')
    df_texts = df_texts.fillna('')

    # Make sure all ids are found in all sets
    ids = list(
        set(df_feature_vector.id.to_list()) &
        set(df_feature_received_by.id.to_list()) &
        set(df_feature_title.id.to_list()) &
        set(df_feature_omk.id.to_list()) &
        set(df_texts.id.to_list()) &
        set(df_responsible.id.to_list()) &
        set(df_time.id.to_list())
    )
    ids = np.unique(np.array(ids))

    # Make sure all ids are found in all sets
    df_time = df_time[df_time.id.isin(ids)]
    df_responsible = df_responsible[df_responsible.id.isin(ids)]
    df_feature_vector = df_feature_vector[df_feature_vector.id.isin(ids)]
    df_feature_received_by = df_feature_received_by[df_feature_received_by.id.isin(ids)]
    df_feature_title = df_feature_title[df_feature_title.id.isin(ids)]
    df_feature_omk = df_feature_omk[df_feature_omk.id.isin(ids)]
    df_texts = df_texts[df_texts.id.isin(ids)]

    # Drop duplicates
    df_time = df_time.drop_duplicates(subset='id')
    df_responsible = df_responsible.drop_duplicates(subset='id')
    df_feature_vector = df_feature_vector.drop_duplicates(subset='id')
    df_feature_received_by = df_feature_received_by.drop_duplicates(subset='id')
    df_feature_title = df_feature_title.drop_duplicates(subset='id')
    df_feature_omk = df_feature_omk.drop_duplicates(subset='id')
    df_texts = df_texts.drop_duplicates(subset='id')

    # Sort by id
    df_time = df_time.sort_values(by='id')
    df_responsible = df_responsible.sort_values(by='id')
    df_feature_vector = df_feature_vector.sort_values(by='id')
    df_feature_received_by = df_feature_received_by.sort_values(by='id')
    df_feature_title = df_feature_title.sort_values(by='id')
    df_feature_omk = df_feature_omk.sort_values(by='id')
    df_texts = df_texts.sort_values(by='id')

    # Reset index for iteration purposes
    df_time = df_time.reset_index(drop=True)
    df_responsible = df_responsible.reset_index(drop=True)
    df_feature_vector = df_feature_vector.reset_index(drop=True)
    df_feature_received_by = df_feature_received_by.reset_index(drop=True)
    df_feature_title = df_feature_title.reset_index(drop=True)
    df_feature_omk = df_feature_omk.reset_index(drop=True)
    df_texts = df_texts.reset_index(drop=True)

    # ----------------------------------------------------------------------------------------------------------------------

    arr_time = np.random.rand(len(df_time), 100)
    for iy, ix in np.ndindex(arr_time.shape):
        arr_time[iy, ix] = 0 # (arr_time[iy, ix] / 2) + 0.5

    for i, el in df_time.iterrows():
        index = int(df_responsible.iloc[i].label_encoded)
        if float(el.label) == 0.0:
            arr_time[i][index] = 5.0
        if float(el.label) == 1.0:
            arr_time[i][index] = 4.0
        if float(el.label) == 2.0:
            arr_time[i][index] = 3.0
        if float(el.label) == 1.0:
            arr_time[i][index] = 2.0
        if float(el.label) == 0.0:
            arr_time[i][index] = 1.0

    # ----------------------------------------------------------------------------------------------------------------------

    arr_responsible = df_responsible[['label_encoded']].to_numpy()
    arr_texts = df_texts[['text']].to_numpy()

    # ----------------------------------------------------------------------------------------------------------------------

    arr_feature_vector = df_feature_vector.drop(['id'], axis=1)
    arr_feature_received_by = df_feature_received_by.drop(['id'], axis=1)
    arr_feature_title = df_feature_title.drop(['id'], axis=1)
    arr_feature_omk = df_feature_omk.drop(['id'], axis=1)

    arr_feature_vector = arr_feature_vector.values
    arr_feature_received_by = arr_feature_received_by.values
    arr_feature_title = arr_feature_title.values
    arr_feature_omk = arr_feature_omk.values

    nrows = 10000
    idx = None

    if nrows is not None:
        idx = random.sample(range(0, len(arr_feature_vector)), nrows)

    if idx is not None:
        arr_feature_vector = [arr_feature_vector[i] for i in idx]
        arr_feature_received_by = [arr_feature_received_by[i] for i in idx]
        arr_feature_title = [arr_feature_title[i] for i in idx]
        arr_feature_omk = [arr_feature_omk[i] for i in idx]
        arr_texts = [arr_texts[i] for i in idx]
        arr_time = [arr_time[i] for i in idx]
        arr_responsible = [arr_responsible[i] for i in idx]

    arr_feature_vector = np.asarray(arr_feature_vector)
    arr_feature_received_by = np.asarray(arr_feature_received_by)
    arr_feature_title = np.asarray(arr_feature_title)
    arr_feature_omk = np.asarray(arr_feature_omk)
    arr_texts = np.asarray(arr_texts).reshape(-1)

    arr_time = np.asarray(arr_time)
    arr_responsible = np.asarray(arr_responsible)

    train_feature_vector, \
    validation_feature_vector, \
    train_received_by, \
    validation_received_by, \
    train_title, \
    validation_title, \
    train_omk, \
    validation_omk, \
    train_texts, \
    validation_texts, \
    train_time, \
    validation_time, \
    train_responsible, \
    validation_responsible \
        = train_test_split(
        arr_feature_vector.tolist(),
        arr_feature_received_by.tolist(),
        arr_feature_title.tolist(),
        arr_feature_omk.tolist(),
        arr_texts.tolist(),
        arr_time.tolist(),
        arr_responsible.tolist(),
        test_size=.2,
        shuffle=True
    )

    tokenized_train = defaultdict(list)
    tokenized_train['feature_vector'] = tf.convert_to_tensor(train_feature_vector)
    tokenized_train['feature_received_by'] = tf.convert_to_tensor(train_received_by)
    tokenized_train['feature_title'] = tf.convert_to_tensor(train_title)
    tokenized_train['feature_omk'] = tf.convert_to_tensor(train_omk)

    tokenized_train['text.input_ids'] = dict(tokenize_texts(train_texts))['input_ids']
    tokenized_train['text.attention_mask'] = dict(tokenize_texts(train_texts))['attention_mask']

    tokenized_train_y = defaultdict(list)
    tokenized_train_y['time'] = tf.convert_to_tensor(train_time)

    tokenized_validation = defaultdict(list)
    tokenized_validation['feature_vector'] = tf.convert_to_tensor(validation_feature_vector)
    tokenized_validation['feature_received_by'] = tf.convert_to_tensor(validation_received_by)
    tokenized_validation['feature_title'] = tf.convert_to_tensor(validation_title)
    tokenized_validation['feature_omk'] = tf.convert_to_tensor(validation_omk)

    tokenized_validation['text.input_ids'] = dict(tokenize_texts(validation_texts))['input_ids']
    tokenized_validation['text.attention_mask'] = dict(tokenize_texts(validation_texts))['attention_mask']

    tokenized_validation_y = defaultdict(list)
    tokenized_validation_y['time'] = tf.convert_to_tensor(validation_time)

    X = tf.data.Dataset.from_tensor_slices((
        dict(tokenized_train),
        dict(tokenized_train_y),
    )).batch(batch_size).prefetch(1)

    V = tf.data.Dataset.from_tensor_slices((
        dict(tokenized_validation),
        dict(tokenized_validation_y),
    )).batch(batch_size).prefetch(1)

    return X, V, \
           len(tokenized_train['feature_vector'][0]), \
           len(tokenized_train['feature_received_by'][0]), \
           len(tokenized_train['feature_title'][0]), \
           len(tokenized_train['feature_omk'][0]), \
           len(tokenized_train['text.input_ids'][0])

In [30]:
X, V, \
len_feature_vector, \
len_feature_received_by, \
len_feature_title, \
len_feature_omk, \
len_text, \
    = data()

In [31]:
input_ids_texts, attention_mask_texts, output_texts = get_layer_for_texts()

All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

All the layers of TFXLMRobertaForSequenceClassification were initialized from the model checkpoint at models/XLM-RoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.


In [39]:
# Input
feature_vector_input = tf.keras.layers.Input(shape=(len_feature_vector,), dtype=tf.float64, name='feature_vector')
feature_received_by_input = tf.keras.layers.Input(shape=(len_feature_received_by,), dtype=tf.float64, name='feature_received_by')
feature_title_input = tf.keras.layers.Input(shape=(len_feature_title,), dtype=tf.float64, name='feature_title')
feature_omk_input = tf.keras.layers.Input(shape=(len_feature_omk,), dtype=tf.float64, name='feature_omk')

# Concatenate
features_concat_time = tf.keras.layers.Concatenate()([
    feature_vector_input,
    feature_received_by_input,
    feature_title_input,
    feature_omk_input
])

output_time = tf.keras.layers.Dense(
    units=len_feature_vector + len_feature_received_by + len_feature_title + len_feature_omk,
    activation='relu'
)(features_concat_time)

output_time = tf.keras.layers.Dropout(rate=.1)(output_time)

output_time = tf.keras.layers.Dense(
    units=len_feature_vector + len_feature_received_by + len_feature_title + len_feature_omk,
    activation='relu'
)(output_time)

output_time = tf.keras.layers.Dropout(rate=.1)(output_time)
output_time = tf.keras.layers.Dense(units=100, activation='relu')(output_time)

output = tf.keras.layers.concatenate([output_time, output_texts])
output = tf.keras.layers.Dense(units=100, name='time')(output)

model = tf.keras.Model(
    inputs=[
        feature_vector_input,
        feature_received_by_input,
        feature_title_input,
        feature_omk_input,
        input_ids_texts,
        attention_mask_texts
    ],
    outputs=[output],
)

In [40]:
batches_per_epoch = 100 // batch
total_train_steps = int(batches_per_epoch * epochs)
optimizer, _ = create_optimizer(init_lr=2e-5, num_warmup_steps=10000, num_train_steps=total_train_steps)

In [41]:
model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer='Adamax',
    metrics=['mse'],
)

model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 feature_vector (InputLayer)    [(None, 5)]          0           []                               
                                                                                                  
 feature_received_by (InputLaye  [(None, 312)]       0           []                               
 r)                                                                                               
                                                                                                  
 feature_title (InputLayer)     [(None, 910)]        0           []                               
                                                                                                  
 feature_omk (InputLayer)       [(None, 122)]        0           []                         

In [42]:
model.fit(
    x=X,
    y=None,
    epochs=epochs,
    validation_data=V,
)

model.save('models/XLM-RoBERTa Ensemble')

 46/500 [=>............................] - ETA: 3:23 - loss: 1.4390 - mse: 1.4390   

KeyboardInterrupt: 